In [10]:
import pandas as pd
import plotly.express as px
from datetime import datetime
from pathlib import Path
from IPython.display import display
from typing import List, Tuple
import glob

class Config:
    PADRAO_ARQUIVO_ENTRADA: str = "resumo_comparativo_*.csv"
    PREFIXO_ARQUIVO_SAIDA: str = "unificado"

    COLUNA_TEMPO: str = 'Execution_Time'
    COLUNAS_IDENTIFICADORAS: List[str] = ['Job_Name', 'Dataset']

    METRICAS_PERFORMANCE: List[str] = ['mAP50_95', 'mAP50', 'Precision', 'Recall', 'F1_Score']
    METRICAS_CUSTO: List[str] = ['Training_Time_Min', 'Latency_ms']

    @classmethod
    def colunas_todas_metricas(cls) -> List[str]:
        return cls.METRICAS_PERFORMANCE + cls.METRICAS_CUSTO

def carregar_dados(padrao_busca: str, prefixo_excluir: str) -> Tuple[List[pd.DataFrame], List[Path]]:
    caminhos = [Path(p) for p in glob.glob(padrao_busca)]
    arquivos_validos = [p for p in caminhos if not p.name.startswith(prefixo_excluir)]

    if not arquivos_validos:
        print("[AVISO] Nenhum arquivo CSV para análise foi encontrado.")
        return [], []

    print(f"Arquivos encontrados para unificação: {[p.name for p in arquivos_validos]}")
    dataframes, arquivos_lidos = [], []
    for caminho in arquivos_validos:
        try:
            df = pd.read_csv(caminho)
            try:
                timestamp_str = '_'.join(caminho.stem.split('_')[-2:])
                df[Config.COLUNA_TEMPO] = datetime.strptime(timestamp_str, '%d-%m-%Y_%H-%M-%S')
            except (ValueError, IndexError):
                df[Config.COLUNA_TEMPO] = datetime.fromtimestamp(caminho.stat().st_mtime)
            dataframes.append(df)
            arquivos_lidos.append(caminho)
        except Exception as e:
            print(f"[ERRO] Falha ao ler o arquivo {caminho.name}: {e}")
    return dataframes, arquivos_lidos

def preparar_dataframe(dfs: List[pd.DataFrame]) -> pd.DataFrame:
    if not dfs: return pd.DataFrame()
    df = pd.concat(dfs, ignore_index=True)
    linhas_iniciais = len(df)
    df.drop_duplicates(inplace=True)

    for col in Config.colunas_todas_metricas():
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(subset=Config.colunas_todas_metricas(), inplace=True)

    print(f"Processamento concluído: {linhas_iniciais - len(df)} linhas inválidas removidas. Total de {len(df)} linhas válidas.")
    return df.sort_values(Config.COLUNA_TEMPO).reset_index(drop=True)

def apresentar_tabela_consolidada(df: pd.DataFrame) -> None:
    print("\n" + "="*80 + "\nPASSO 2: RESULTADOS QUANTITATIVOS CONSOLIDADOS\n" + "="*80)
    try:
        tabela_pivot = df.pivot_table(index='Job_Name', columns='Dataset', values=Config.METRICAS_PERFORMANCE)
        tabela_pivot_formatada = tabela_pivot.swaplevel(0, 1, axis=1)
        tabela_pivot_formatada.sort_index(axis=1, level=0, inplace=True)

        styled_table = tabela_pivot_formatada.style.format("{:.4f}") \
                .highlight_max(color='#D4EDDA', axis=0) \
                .set_caption("Tabela 2: Análise Quantitativa Detalhada do Desempenho dos Modelos nos Datasets de Avaliação.")
        display(styled_table)
    except Exception as e:
        print(f"[ERRO] Não foi possível gerar a tabela consolidada: {e}")

def apresentar_destaques(df: pd.DataFrame) -> None:
    print("\n" + "="*80 + "\nPASSO 3: SÍNTESE DOS MELHORES RESULTADOS\n" + "="*80)

    destaques_data = {
        'Melhor mAP@.50-.95': df.loc[df['mAP50_95'].idxmax()],
        'Melhor Precisão': df.loc[df['Precision'].idxmax()],
        'Melhor Recall': df.loc[df['Recall'].idxmax()],
        'Menor Latência (Inferência)': df.loc[df['Latency_ms'].idxmin()]
    }

    df_destaques = pd.DataFrame(destaques_data).T[Config.COLUNAS_IDENTIFICADORAS + Config.colunas_todas_metricas()]
    display(df_destaques.style.set_caption("Tabela 3: Síntese dos modelos de melhor desempenho para as principais métricas.") \
            .format(precision=4).background_gradient(cmap='viridis', axis=0))

def apresentar_visualizacoes(df: pd.DataFrame) -> None:
    print("\n" + "="*80 + "\nPASSO 4: ANÁLISE GRÁFICA COMPARATIVA\n" + "="*80)

    df_melt_map = df.melt(id_vars=Config.COLUNAS_IDENTIFICADORAS, value_vars=['mAP50_95', 'mAP50'], var_name='Métrica', value_name='Valor')
    fig1 = px.bar(df_melt_map, x='Job_Name', y='Valor', color='Métrica', barmode='group', facet_col='Dataset',
                  title='<b>Figura 1: Análise Comparativa do Mean Average Precision (mAP) por Modelo e Dataset</b>',
                  labels={'Valor': 'Valor mAP', 'Job_Name': 'Modelo'}, template='simple_white')
    fig1.update_yaxes(rangemode='tozero', title_text='mAP')
    fig1.show()

    df_melt_pr = df.melt(id_vars=Config.COLUNAS_IDENTIFICADORAS, value_vars=['Precision', 'Recall'], var_name='Métrica', value_name='Valor')
    fig2 = px.bar(df_melt_pr, x='Job_Name', y='Valor', color='Métrica', barmode='group', facet_col='Dataset',
                  title='<b>Figura 2: Análise da Relação entre Precisão e Recall por Modelo e Dataset</b>',
                  labels={'Valor': 'Valor da Métrica', 'Job_Name': 'Modelo'}, template='simple_white')
    fig2.update_yaxes(rangemode='tozero')
    fig2.show()

    df_melt_custo = df.melt(id_vars=Config.COLUNAS_IDENTIFICADORAS, value_vars=Config.METRICAS_CUSTO, var_name='Métrica', value_name='Valor')
    fig3 = px.bar(df_melt_custo, x='Job_Name', y='Valor', color='Métrica', barmode='group', facet_col='Dataset',
                  title='<b>Figura 3: Análise Comparativa da Eficiência Computacional (Treinamento e Inferência)</b>',
                  labels={'Valor': 'Valor (unidade específica)', 'Job_Name': 'Modelo'}, template='simple_white')
    fig3.update_yaxes(rangemode='tozero')
    fig3.show()

def salvar_dados_unificados(df: pd.DataFrame, arquivos_originais: List[Path], prefixo: str) -> None:
    nomes_base = '-'.join([p.stem for p in arquivos_originais])
    nome_alternativo = f"de_{len(arquivos_originais)}_arquivos_unificados"

    if len(prefixo) + len(nomes_base) > 150:
        print(f"[AVISO] O nome do arquivo gerado seria muito longo. Usando um nome simplificado.")
        nomes_base = nome_alternativo

    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    nome_arquivo = f"{prefixo}_{nomes_base}_{timestamp}.csv"

    try:
        df.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')
        print("\n" + "="*80 + f"\n[SUCESSO] Arquivo consolidado salvo como: '{nome_arquivo}'\n" + "="*80)
    except Exception as e:
        print(f"\n[ERRO CRÍTICO] Falha ao salvar o arquivo unificado: {e}")

if __name__ == "__main__":
    print("="*80 + "\nINICIANDO A AVALIAÇÃO EXPERIMENTAL COMPARATIVA\n" + "="*80)

    lista_dfs, arquivos_processados = carregar_dados(Config.PADRAO_ARQUIVO_ENTRADA, Config.PREFIXO_ARQUIVO_SAIDA)
    df_final = preparar_dataframe(lista_dfs)

    if not df_final.empty:
        apresentar_tabela_consolidada(df_final)
        apresentar_destaques(df_final)
        apresentar_visualizacoes(df_final)
        salvar_dados_unificados(df_final, arquivos_processados, Config.PREFIXO_ARQUIVO_SAIDA)
    else:
        print("\n" + "="*80 + "\nANÁLISE INTERROMPIDA: Nenhum dado válido foi encontrado.\n" + "="*80)

INICIANDO A AVALIAÇÃO EXPERIMENTAL COMPARATIVA
Arquivos encontrados para unificação: ['resumo_comparativo_21-09-2025_09-35-31-005.csv', 'resumo_comparativo_21-09-2025_07-31-36.csv', 'resumo_comparativo_20-09-2025_00-01-13.csv', 'resumo_comparativo_21-09-2025_10-49-22-009.csv', 'resumo_comparativo_21-09-2025_13-23-51.csv', 'resumo_comparativo_20-09-2025_20-29-25.csv']
Processamento concluído: 0 linhas inválidas removidas. Total de 14 linhas válidas.

PASSO 2: RESULTADOS QUANTITATIVOS CONSOLIDADOS



PASSO 3: SÍNTESE DOS MELHORES RESULTADOS


,Job_Name,Dataset,mAP50_95,mAP50,Precision,Recall,F1_Score,Training_Time_Min,Latency_ms
Melhor mAP@.50-.95,YOLO11n_640px_100e,MegaFaunaSplit,0.7886,0.9631,0.9426,0.9104,0.9262,77.9720,0.0000
Melhor Precisão,YOLOv5n_640px_100e,MegaFaunaSplit,0.7876,0.9657,0.9455,0.9171,0.9311,90.1807,0.0000
Melhor Recall,YOLOv5n_640px_100e,MegaFaunaSplit,0.7876,0.9657,0.9455,0.9171,0.9311,90.1807,0.0000
Menor Latência (Inferência),YOLOv5n_640px_100e,aquarium_pretrain,0.3968,0.7106,0.7171,0.6762,0.6960,6.9234,0.0000



PASSO 4: ANÁLISE GRÁFICA COMPARATIVA


[AVISO] O nome do arquivo gerado seria muito longo. Usando um nome simplificado.

[SUCESSO] Arquivo consolidado salvo como: 'unificado_de_6_arquivos_unificados_2025-09-21_13-55-01.csv'
